# Eat Safe, Love

## Notebook Set Up

In [1]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint




In [2]:
# Create an instance of MongoClient
mongo = MongoClient("localhost",port=27017) 

In [3]:
db = mongo['uk_food']

# assign the uk_food database to a variable name
db = mongo['uk_food']
print(db)

In [4]:
# review the collections in our database
db.list_collection_names()

ServerSelectionTimeoutError: localhost:27017: [Errno 61] Connection refused (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30s, Topology Description: <TopologyDescription id: 65d3112a8605a31ee1e3b2ce, topology_type: Unknown, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [Errno 61] Connection refused (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>]>

In [ ]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [5]:
# Find the establishments with a hygiene score of 20
query = {'scores.Hygiene': {'$eq': 20}}

# Use count_documents to display the number of documents in the result
hygiene = establishments.count_documents(query)
# Display the first document in the results using pprint
result = establishments.find(query)
pprint(result[0])

NameError: name 'establishments' is not defined

In [ ]:
# Convert the result to a Pandas DataFrame
PDataFrame = pd.DataFrame(establishments.find(query))
# Display the number of rows in the DataFrame
print(f"There are {len(PDataFrame)} rows in this DataFrame.")
# Display the first 10 rows of the DataFrame
PDataFrame.head(10)

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [ ]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query2 = {'LocalAuthorityName': {'$regex': 'London'}, 
          'RatingValue': {'$gte': '4'}
         }

# Use count_documents to display the number of documents in the result
print(f'There are {establishments.count_documents(query2)} establishments in London with a Rating Value of 4 or more')
# Display the first document in the results using pprint
result1 = establishments.find(query2)
pprint(result1[0])

In [ ]:
# Convert the result to a Pandas DataFrame
PDataFrame2 = pd.DataFrame(establishments.find(query2))
# Display the number of rows in the DataFrame
print(f"There are {len(PDataFrame2)} rows in this DataFrame.")
# Display the first 10 rows of the DataFrame
PDataFrame2.head(10)

### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [ ]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = 51.490142
longitude = 0.08384

query3 = {'RatingValue': '5', 
'geocode.latitude': {'$lte': (latitude + degree_search), '$gte': (latitude - degree_search)}, 
'geocode.longitude': {'$lte': (longitude + degree_search), '$gte': (longitude - degree_search)} 
}
sort =  [('scores.Hygiene', 1)]
limit = 5
query3_result = list(establishments.find(search).sort(sort).limit(limit))
# Print the results
pprint(query3_result)

In [ ]:
# Convert result to Pandas DataFrame
PDataFrame3 = pd.DataFrame(establishments.find(query3).sort(sort).limit(5))
PDataFrame3

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [ ]:
# Create a pipeline that:
# 1. Matches establishments with a hygiene score of 0
match_query = {'$match': {'scores.Hygiene' : 0}}
# 2. Groups the matches by Local Authority
group_query = {'$group': {'_id' : '$LocalAuthorityName', 'count':{'$sum':1}}}
# 3. Sorts the matches from highest to lowest
sort = {'$sort': {'count': -1}}
pipeline = [match_query, group_query, sort]
# Print the number of documents in the result
results = list(establishments.aggregate(pipeline))
# Print the first 10 results
pprint(results[0:10])

In [ ]:
# Convert the result to a Pandas DataFrame
hygiene_df = pd.json_normalize(results)
# Display the number of rows in the DataFrame
print(f"There are {len(hygiene_df)} rows in the DataFrame.")
# Display the first 10 rows of the DataFrame
hygiene_df.head(10)